In [30]:
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import re
import numpy as np
import os

In [31]:
# ChromeDriver 시작 함수
def start_chromedriver():
    service = Service("/opt/homebrew/bin/chromedriver")  # chromedriver 경로 수정 필요
    driver = webdriver.Chrome(service=service)
    print("Chrome 드라이버 실행.")
    return service, driver

# ChromeDriver 종료 함수
def end_chromedriver(driver, service):
    driver.quit()
    print("Chrome 드라이버를 닫음.")
    if service:
        service.stop()
        print("ChromeDriver 서비스 종료.")

In [55]:
def get_urls():
    # 무신사 랭킹 페이지 URL
    ranking_urls = {
        "clothes_top": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=199&categoryCode=001000&period=MONTHLY",
        "outers": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=199&categoryCode=002000&period=MONTHLY",
        "pants": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=199&categoryCode=003000&period=MONTHLY",
        "shoes": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=199&categoryCode=103000&period=MONTHLY",
    }
    return ranking_urls

In [56]:
def extract_product_details(driver, product_id, date):
    try:
        # 상품 상세 페이지로 이동
        product_url = f"https://www.musinsa.com/products/{product_id}"
        print(f"접근 중: {product_url}")
        driver.get(product_url)
        # time.sleep(3)  # 페이지 로드 대기


        # HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        # 조회수 추출
        views = None
        views_label = soup.find("span", string="조회수")  # `text`를 `string`으로 변경
        if views_label:
            views_element = views_label.find_next("span")
            if views_element:
                views = views_element.text.strip()
                
        # 총 판매량 추출
        total_sales = None
        total_sales_label = soup.find("span", string="누적판매")  # `text`를 `string`으로 변경
        if total_sales_label:
            total_sales_element = total_sales_label.find_next("span")
            if total_sales_element:
                total_sales = total_sales_element.text.strip()
                
        # 좋아요 수 추출
        likes = None
        like_button_div = soup.find("div", {"aria-label": "좋아요 버튼"})
        if like_button_div:
            parent_button = like_button_div.find_parent("button")
            if parent_button:
                sibling_spans = parent_button.find_all_next("span", {"class": "text-xs font-medium font-pretendard"})
                for sibling_span in sibling_spans:
                    span_text = sibling_span.text.strip()

                    if span_text.replace(",", "").isdigit():
                        likes = int(span_text.replace(",", ""))
                        break

        rating = None
        rating_count = None

        rating_element = soup.select_one("span.font-medium.text-black")
        if rating_element:
            rating_text = rating_element.text.strip()
            if rating_text.replace(".", "", 1).isdigit():  # 소수점 허용
                rating = float(rating_text)  # 평점은 소수로 변환

        if rating_element:
            sibling_element = rating_element.find_next_sibling("span")
            if sibling_element and sibling_element.text.strip():
                rating_count = extract_number_value(sibling_element.text.strip())

            
        # 텍스트를 숫자로 변환
        views = extract_number_value(views) if views else None
        total_sales = extract_number_value(total_sales) if total_sales else None
        
        # 반환
        details = {
            "ProductID": product_id,
            "Date": date,
            "Views": views,
            "Likes": likes,
            "Rating": rating,
            "RatingCount": rating_count, 
            "TotalSales": total_sales,
        }
        print(f"추출된 데이터: {details}")
        return details
    except Exception as e:
        print(f"상품 세부 정보 추출 오류 (ID: {product_id}): {e}")
        return None

In [57]:
def extract_number_value(text):
        if isinstance(text, float) and np.isnan(text):
            return 0
        if isinstance(text, list):
            text = text[0] if text else ""
        if not isinstance(text, str):
            text = str(text)

        # 쉼표 제거
        text = text.replace(",", "").replace(" ", "")

        match = re.search(r"\d+(\.\d+)?", text)
        if not match:
            return np.nan
        number = float(match.group())
        if "백" in text:
            number *= 100
        if "천" in text:
            number *= 1000
        if "만" in text:
            number *= 10000
        return int(number)

In [67]:
def process_csv_file(input_csv, output_dir, driver, category, target_date):
    """CSV 파일 읽기 및 처리 로직"""
    try:
        df = pd.read_csv(input_csv)

        # 열 이름 출력
        print(f"열 이름: {list(df.columns)}")

        # 'productId'와 'date' 열 확인
        if "productId" in df.columns and "date" in df.columns:
            print(f"'{input_csv}'에 'productId'와 'date' 열이 있습니다.")
        else:
            print(f"'{input_csv}'에 'productId' 또는 'date' 열이 없습니다.")
            return  # 다음 파일로 넘어감

        # 'date' 열을 숫자형으로 변환
        df['date'] = pd.to_numeric(df['date'], errors='coerce')

        # NaN 값 제거
        df = df.dropna(subset=['date'])

        # 'date'를 정수형으로 변환
        df['date'] = df['date'].astype(int)

        # 특정 날짜 필터링
        filtered_df = df[df['date'] == target_date]
        if filtered_df.empty:
            print(f"'{input_csv}'에서 {target_date}에 해당하는 데이터가 없습니다.")
            return  # 다음 파일로 넘어감

        product_ids = filtered_df["productId"]
        dates = filtered_df["date"]

        # 상세 데이터 수집
        detailed_data = []
        for product_id, date in zip(product_ids, dates):
            details = extract_product_details(driver, product_id, date)
            if details:
                detailed_data.append(details)

        # 결과를 출력 디렉토리에 저장
        output_csv = os.path.join(output_dir, f"detail_{category}.csv")
        detailed_df = pd.DataFrame(detailed_data)
        detailed_df.to_csv(output_csv, index=False, encoding="utf-8-sig")
        print(f"{output_csv}에 저장 완료.")

    except Exception as e:
        print(f"파일 처리 중 오류 발생: {e}")

In [68]:
def process_detailed_data(driver, input_dir, output_dir, target_date):
    # 출력 디렉토리 생성
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 입력 디렉토리 탐색 (폴더 및 파일 모두 처리 가능)
    for item in os.listdir(input_dir):
        item_path = os.path.join(input_dir, item)
        
        if os.path.isdir(item_path):  # 디렉토리인 경우 처리
            for csv_file in os.listdir(item_path):
                if csv_file.endswith(".csv"):  # CSV 파일만 처리
                    input_csv = os.path.join(item_path, csv_file)
                    
                    # 처리 중인 파일 출력
                    print(f"처리 중인 파일(폴더 내): {input_csv}")
                    
                    # CSV 읽기 및 처리
                    process_csv_file(input_csv, output_dir, driver, item, target_date)

        elif item.endswith(".csv"):  # 루트에 바로 CSV 파일이 있는 경우 처리
            input_csv = item_path
            
            # 처리 중인 파일 출력
            print(f"처리 중인 파일(루트): {input_csv}")
            
            # CSV 읽기 및 처리
            process_csv_file(input_csv, output_dir, driver, "root", target_date)  # "root"는 카테고리 이름 대체용

In [69]:
def main():
    input_dir = "data/raw"
    output_dir = "data/raw"
    target_date = 2024120120  # 원하는 날짜 설정
    service, driver = start_chromedriver()
    try:
        process_detailed_data(driver, input_dir, output_dir, target_date)
    finally:
        end_chromedriver(driver, service)


if __name__ == "__main__":
    main()

Chrome 드라이버 실행.
처리 중인 파일(루트): data/raw/ranking-shoes-summary.csv
열 이름: ['productId', 'brandName', 'productName', 'price', 'discountRate', 'currentlyViewing', 'currentlyBuying', 'ranking', 'trending', 'colors', 'date', 'category']
'data/raw/ranking-shoes-summary.csv'에 'productId'와 'date' 열이 있습니다.
접근 중: https://www.musinsa.com/products/4371012
추출된 데이터: {'ProductID': 4371012, 'Date': 2024120120, 'Views': 120000, 'Likes': 12335, 'Rating': 4.9, 'RatingCount': 1073, 'TotalSales': 4400}
접근 중: https://www.musinsa.com/products/4589098
추출된 데이터: {'ProductID': 4589098, 'Date': 2024120120, 'Views': 20000, 'Likes': 1155, 'Rating': 5.0, 'RatingCount': 1, 'TotalSales': None}
접근 중: https://www.musinsa.com/products/4507335
추출된 데이터: {'ProductID': 4507335, 'Date': 2024120120, 'Views': 330000, 'Likes': 16886, 'Rating': 4.9, 'RatingCount': 531, 'TotalSales': 8200}
접근 중: https://www.musinsa.com/products/4328804
추출된 데이터: {'ProductID': 4328804, 'Date': 2024120120, 'Views': 140000, 'Likes': 16744, 'Rating': 4.9